In [51]:
import scipy.stats as sts
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.integrate import quad
from functools import lru_cache
from time import time

np.random.seed(0)
def timer_func(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func
  
  


# Estimators
def nonParametricOptimalQuantity(distributionData, tau):
    optimalQuantity = np.quantile(distributionData,tau)
    return optimalQuantity

def parametricOptimalQuantity(tau, dist, parameters):
    optimalQuantity = dist.ppf(tau, *parameters)
    return optimalQuantity

# Expected profit



def int_dist(y,dist):
    f = dist.pdf(y)
    return y * f
@timer_func
def integrate(Q_star,dist):
    return quad(int_dist,0,Q_star,args=(dist,))[0]

def calculateOptimalProfit(tau,Q_star,dist):
    price = 1
    cost = 1 - tau

    integral_value = integrate(Q_star, dist)
    t1 = price * integral_value
    t2 = price * Q_star * dist.sf(Q_star)
    t3 = cost * Q_star
    return t1 + t2 - t3

# Evaluation of test statistics functions
def empericalRootMeanSquaredError(m, optimalQuantities, realOptimalQuantity):
    rmse = np.sqrt(1/m * np.sum((optimalQuantities - realOptimalQuantity)**2))
    return rmse

def empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, realOptimalQuantity):
    return empericalRootMeanSquaredError(m, nonParametricOptimalQuantitys, realOptimalQuantity) / empericalRootMeanSquaredError(m, parametricOptimalQuantitys, realOptimalQuantity)

def empericalProfitLoss(m, profitEstimator, profitReal):
    return 1 / m * np.sum(np.abs((profitReal - profitEstimator) /  profitReal))

def empericalProfitLossRatio(m, parametricExpectedProfit, nonParametricExpectedProfit, realExpectedProfit):
    return empericalProfitLoss(m, nonParametricExpectedProfit, realExpectedProfit) / empericalProfitLoss(m, parametricExpectedProfit, realExpectedProfit)

def empericalServiceLevel(m, optimalQuantities, demand):
    indicatorFunction = np.where(optimalQuantities.T >= np.array(demand), 1, 0)
    return 1 / m * np.sum(indicatorFunction)

def fit(feature):
    return np.mean(feature), np.std(feature)


def monteCarlo(m, tau, n, meanDemand, stdevDemand):
    # Define arrays
    dist = sts.norm
    distReal = sts.norm(loc=meanDemand, scale=stdevDemand)

    realOptimalQuantity = np.empty((m,1))
    parametricOptimalQuantitys = np.empty((m,1))
    nonParametricOptimalQuantitys = np.empty((m,1))
    allParameters = np.empty((m,2))
    parametricExpectedProfits = np.empty((m,1))
    nonParametricExpectedProfits = np.empty((m,1))


    # Compute estimators and optimal quantities
    for j in range(m):
        distributionData = sts.norm.rvs(loc = meanDemand, scale = stdevDemand, size = n)
        allParameters[j] = parameters = fit(distributionData)

        parametricOptimalQuantitys[j] = parametricOptimalQuantity(tau, dist, parameters)
        nonParametricOptimalQuantitys[j] = nonParametricOptimalQuantity(distributionData, tau)
        
        parametricExpectedProfits[j] = calculateOptimalProfit(tau, parametricOptimalQuantitys[j], distReal)
        nonParametricExpectedProfits[j] = calculateOptimalProfit(tau, nonParametricOptimalQuantitys[j], distReal)
        
    realOptimalQuantity = parametricOptimalQuantity(tau, dist, [meanDemand,stdevDemand])
    realExpectedProfit = calculateOptimalProfit(tau, realOptimalQuantity, distReal)

    eplr = empericalProfitLossRatio(m, parametricExpectedProfits, nonParametricExpectedProfits, realExpectedProfit)

    # Compute evaluation statistics
    rmse = empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, realOptimalQuantity)

    #eslParametric = empericalServiceLevel(m, parametricOptimalQuantitys, demand)
    #eslNonParametric = empericalServiceLevel(m, nonParametricOptimalQuantitys, demand)
    #eplr = empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)
    result = {
                    'MonteCarlo iterations' : m,
                    'Sample Size': n,
                    'Target Service Level': tau,
                    'Param Values': np.mean(allParameters, axis=0),
                    'real optimal quantity' : np.mean(realOptimalQuantity),
                    'Parm optimal quantity': np.mean(parametricOptimalQuantitys),
                    'nonParm optimal quantity': np.mean(nonParametricOptimalQuantitys),
                    'RMSE Ratio': rmse,
                    'SL nonParam': 0,
                    'SL Param': 0,
                    'EPLR': eplr
                }
    return result





In [52]:
numberOfMontecarloIterations = 10
stdevDemand = 15
meanDemand = 120

tauArray = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
nArray = [10, 50, 100, 200]

results = []
for tau in tauArray:
    for n in nArray:
        result = monteCarlo(numberOfMontecarloIterations, tau, n, meanDemand, stdevDemand)
        results.append(result)


Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0140s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0040s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0050s
Function 'integrate' executed in 0.0160s
Function 'integrate' executed in 0.0040s
Function 'integrate' executed in 0.0120s
Function 'integrate' executed in 0.0140s
Function 'integrate' executed in 0.0140s
Function 'integrate' executed in 0.0060s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0120s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0140s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0040s
Function 'integrate' executed in 0.0130s
Function 'integrate' executed in 0.0140s
Function 'integr

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df

,MonteCarlo iterations,Sample Size,Target Service Level,Param Values,real optimal quantity,Parm optimal quantity,nonParm optimal quantity,RMSE Ratio,SL nonParam,SL Param,EPLR
0,10,10,0.01,"[120.89712023301729, 13.11512523315869]",85.104782,90.386777,99.485116,1.405798,0,0,2.900732
1,10,50,0.01,"[118.83146642374027, 14.823948673780814]",85.104782,84.345805,87.039712,2.039607,0,0,7.996710
2,10,100,0.01,"[120.12169789617522, 14.370987797631816]",85.104782,86.689781,90.314428,2.385692,0,0,7.255617
3,10,200,0.01,"[119.49463038004853, 14.656430425316287]",85.104782,85.398675,87.790502,2.799706,0,0,10.106098
4,10,10,0.05,"[119.88595711946206, 13.89606530514327]",95.327196,97.028964,100.259467,1.386976,0,0,2.529137
5,10,50,0.05,"[119.42070034933556, 15.038049227954877]",95.327196,94.685311,97.148857,1.608551,0,0,3.256070
6,10,100,0.05,"[120.29040004397397, 14.923954723116172]",95.327196,95.742679,95.797080,1.352277,0,0,1.774276
7,10,200,0.05,"[119.93274389875894, 14.667436046751032]",95.327196,95.806959,95.740496,1.693983,0,0,2.734982
8,10,10,0.10,"[119.07005882896412, 13.993392417389074]",100.776727,101.136805,105.332131,1.217600,0,0,1.966337
9,10,50,0.10,"[120.54032256185899, 14.994120727370213]",100.776727,101.324584,101.996587,1.253104,0,0,1.745455


In [ ]:
fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher RMSE means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/RMSE-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()

fig = px.scatter(df, x='Target Service Level', y='EPLR', color="Sample Size", title="TSL vs EPLR, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher EPLR means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/EPLR-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()

#fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, higher RMSE means parametric performs better")
#fig.show()


In [ ]:
numberOfMontecarloIterations = 10
stdevDemand = 20
meanDemand = 100

tauArray = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
nArray = [10, 50, 100, 200]

results = []
for tau in tauArray:
    for n in nArray:
        result = monteCarlo(numberOfMontecarloIterations, tau, n, meanDemand, stdevDemand)
        results.append(result)


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df

,MonteCarlo iterations,Sample Size,Target Service Level,Param Values,real optimal quantity,Parm optimal quantity,nonParm optimal quantity,RMSE Ratio,SL nonParam,SL Param,EPLR
0,1000,10,0.01,"[100.05276221108602, 18.322430190433316]",53.473043,57.428416,70.081064,1.612756,0.0,0.000,1.746746
1,1000,50,0.01,"[99.86599774239826, 19.625352516938257]",53.473043,54.210601,58.687906,1.620333,0.0,0.000,1.697920
2,1000,100,0.01,"[99.98793184995574, 19.825710639774723]",53.473043,53.866432,56.936919,1.855067,0.0,0.000,1.909708
3,1000,200,0.01,"[100.01753715293296, 19.93699120840811]",53.473043,53.637160,55.492622,1.876973,0.0,0.000,1.934502
4,1000,10,0.05,"[100.17412834628907, 18.583272451373194]",67.102927,69.607365,74.013078,1.181105,0.0,0.000,1.212582
5,1000,50,0.05,"[100.01290058587084, 19.74449497854405]",67.102927,67.536096,68.846935,1.301301,0.0,0.000,1.307192
6,1000,100,0.05,"[100.13613182339489, 19.7881913695077]",67.102927,67.587453,68.364898,1.318215,0.0,0.000,1.319447
7,1000,200,0.05,"[100.05332981205532, 19.88140049309111]",67.102927,67.351336,67.659238,1.303488,0.0,0.000,1.289564
8,1000,10,0.10,"[100.23942617137259, 18.565601623716454]",74.368969,76.446650,78.927581,1.183254,0.0,0.000,1.187543
9,1000,50,0.10,"[100.04874979208519, 19.739044117267085]",74.368969,74.752147,75.468564,1.272098,0.0,0.000,1.274230


In [ ]:
fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher RMSE means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/RMSE-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()

fig = px.scatter(df, x='Target Service Level', y='EPLR', color="Sample Size", title="TSL vs EPLR, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher EPLR means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/EPLR-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()